# Analysing Onsala-1 radio spectra obtained from OSO 20m telescope

In this notebook, we will analyse a spectrum of Onsala-1 molecular cloud obtained from 20m telescope at OSO. The spectra were measured in radio frequencies and have four emission lines of $CH_3CCH$. The main goal is to find the area under the lines, which is basically the integrated antenna temperature under these lines. With this we can compute the column density of the $CH_3CCH$.

The focus of this notebook is Onsala-1. Once we have worked it out, we can make a routine to compute the same for the rest of the targets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
from glob import glob
import astropy.constants as con
import utils as utl
from scipy.optimize import minimize
import dynesty
from scipy import stats
import pickle
from dynesty import plotting as dyplot
from dynesty.utils import resample_equal
import lmfit
import corner

We first make a list of all fits file which has spectrum of Onsala-1. We also list the system temperature and the total integration time, which we need to compute the combined/average spectrum (to gain a higher S/N).

In [ ]:
# Listing all of the fits file
f1 = glob(os.getcwd() + '/Data/*.fits')
ons, tsys, int_time = np.array([]), np.array([]), np.array([])
print('File\t\tObject\t\tT_sys\tINTTIME')
print('-----------------------------------------------')
for i in range(len(f1)):
    hdul = fits.open(f1[i])
    hdr = hdul[0].header
    if hdr['OBJECT'] == 'Onsala 1':
        ons = np.hstack((ons, f1[i]))
        tsys, int_time = np.hstack((tsys, hdr['TSYS'])),\
             np.hstack((int_time, hdr['INTTIME']))
        print(f1[i].split('/')[-1] + '\t' + hdr['OBJECT'] + '\t'\
             + str(hdr['TSYS']) + '\t' + str(hdr['INTTIME']))

We can now compute an average spectrum out of all of these spectra!

In [ ]:
# For zeroth spectra
hdul5 = fits.open(ons[0])
hdr5, dta5 = hdul5[0].header, hdul5[0].data[0][0]
ii5 = np.arange(hdr5['NAXIS1']) + 1
freq_all = hdr5['RESTFREQ'] + hdr5['CRVAL1'] + hdr5['CDELT1']*(ii5-hdr5['CRPIX1'])
freq_all = int_time[0]*freq_all/tsys[0]
temp_all = int_time[0]*dta5/tsys[0]
# For all other spectra
for i in range(len(ons)-1):
    hdul5 = fits.open(ons[i+1])
    hdr5, dta5 = hdul5[0].header, hdul5[0].data[0][0]
    ii5 = np.arange(hdr5['NAXIS1']) + 1
    frq5 = hdr5['RESTFREQ'] + hdr5['CRVAL1'] + hdr5['CDELT1']*(ii5-hdr5['CRPIX1'])
    # Saving the data
    temp_all = np.vstack((temp_all, int_time[i+1]*dta5/tsys[i+1]))
    freq_all = np.vstack((freq_all, int_time[i+1]*frq5/tsys[i+1]))
# Weighted average (weighted with int_time/Tsys) over all the spectra
freq_avg = np.sum(freq_all, axis=0)/np.sum(int_time/tsys)
temp_avg = np.sum(temp_all, axis=0)/np.sum(int_time/tsys)

# Plotting the result:
plt.figure(figsize=(16/1.5, 9/1.5))
plt.errorbar(freq_avg/1e9, temp_avg, fmt='.', c='orangered')
plt.xlabel('Frequency (in GHz)')
plt.ylabel('Temperature (K)')
plt.xlim([np.min(freq_avg/1e9), np.max(freq_avg/1e9)])
plt.grid()

There we go! This is a nice spectrum of Onsala 1!
We can first convert frequency to the velocity first -- to do this we can use the following Doppler formula:

$$v = c \cdot \frac{f_s - f_o}{f_s}$$

The symbols have their usual meanings.

In [ ]:
# To find the rest frame frequency
diff = freq_avg - hdr5['OBSFREQ']
rest_freq = hdr5['RESTFREQ']#freq_avg + diff
# To find the velocity of the target!
velo_avg = con.c.value*(rest_freq - freq_avg)/rest_freq
velo_avg = velo_avg + hdr5['VLSR']# + hdr5['VELO-GEO'] + hdr5['VELO-HEL']
# We can plot the results
plt.figure(figsize=(16/1.5, 9/1.5))
plt.errorbar(velo_avg/1e3, temp_avg, fmt='.', c='orangered')
plt.xlabel('Velocity (in km/s)')
plt.ylabel('Temperature (in K)')
plt.xlim([np.min(velo_avg/1e3), np.max(velo_avg/1e3)])
plt.grid()

## How to compute the uncertainties on these points

One peculier thing about this spectrum is that its points do not have errors on them. We all know that errors are fundamentals to any spectrum --- but, how do we estimate the unceratainties? One way out could be to evaluate the standard deviation of the _whole_ spectrum and treat it as a stadard deviation of a Gaussian distribution with zero mean, which then makes an array of uncertainties.

We can do this bacause the spctral lines are like outliers to the continumm (see, the histogram of the spectrum below), which is basically a noise.

Below we implement this:

In [ ]:
plt.figure(figsize=(6, 8))
plt.hist(temp_avg, bins=100, histtype='step')
plt.xlabel('Temperature (in K)')
plt.ylabel('Counts')
temp_err = np.abs(np.random.normal(0, np.std(temp_avg), len(temp_avg)))

And this is how the final spectrum would look like...

In [ ]:
# We can plot the results
## Binned data
velo_bin, temp_bin, temp_err_bin, _ = utl.lcbin(velo_avg, temp_avg, binwidth=500)
plt.figure(figsize=(16/1.5, 9/1.5))
plt.errorbar(velo_avg/1e3, temp_avg, yerr=temp_err, fmt='.', c='orangered', alpha=0.1)
plt.errorbar(velo_bin/1e3, temp_bin, yerr=temp_err_bin, fmt='o', mfc='white', c='black')
plt.xlabel('Velocity (in km/s)')
plt.ylabel('Temperature (in K)')
plt.xlim([0, 90])
plt.grid()

## And the fitting...

So, there are four lines (though we cannot see them individually) along with some offset trend. What we can do is to build a `lmfit` model that has four gaussian models and a linear model. Then we can first use `lmfit` to fit it to the data, and then using `emcee` method to explore the posterior probabilities.

In [ ]:
def gaussian(x, mu, sig, amp):
    return amp*np.exp(-0.5 * ((x-mu)/sig)**2)
def line(x,m,c):
    return m*x + c

# Adding our initial guesses
pars = lmfit.Parameters()
pars.add_many(('m', 0., True), ('c', 0.1, True),\
    ('mu1', 11000., True), ('sig1', 1500., True), ('amp1', 0.4, True),\
    ('mu2', 18000., True), ('sig2', 1500., True), ('amp2', 0.3, True),\
    ('mu3', 35000., True), ('sig3', 1500., True), ('amp3', 0.13, True),\
    ('mu4', 62000., True), ('sig4', 1500., True), ('amp4', 0.1, True))

def tot_model(x, p):
    v = p.valuesdict()
    model_fin = gaussian(x, v['mu1'], v['sig1'], v['amp1']) +\
        gaussian(x, v['mu2'], v['sig2'], v['amp2'])+\
        gaussian(x, v['mu3'], v['sig3'], v['amp3'])+\
        gaussian(x, v['mu4'], v['sig4'], v['amp4'])+\
        line(x, v['m'], v['c'])
    return model_fin

def residual(p):
    model_fin = tot_model(velo_avg, p)
    chi2 = model_fin-temp_avg#)/temp_err
    return chi2

def residual_wt(p):
    model_fin = tot_model(velo_avg, p)
    chi2 = (model_fin-temp_avg)/temp_err
    return chi2

We can now use `lmfit.minimize` to find the "first" best-fitted parameters.

In [ ]:
mi = lmfit.minimize(residual, pars, method='nelder', nan_policy='omit')
mi

In [ ]:
vall = np.linspace(np.min(velo_avg), np.max(velo_avg), 10000)
best_fit = tot_model(vall, mi.params)

plt.figure(figsize=(16/1.5, 9/1.5))
plt.errorbar(velo_avg/1e3, temp_avg, yerr=temp_err, fmt='.', c='orangered', alpha=0.1)
plt.errorbar(velo_bin/1e3, temp_bin, yerr=temp_err_bin, fmt='o', mfc='white', c='black')
plt.plot(vall/1e3, best_fit, c='cornflowerblue', lw=3, zorder=5)
plt.xlabel('Velocity (in km/s)')
plt.ylabel('Temperature (in K)')
plt.xlim([0, 90])
plt.grid()

In [ ]:
res = lmfit.minimize(residual_wt, method='emcee', nan_policy='omit', burn=10000, steps=50000, thin=20,
                     params=mi.params, is_weighted=True, progress=True)

In [ ]:
emcee_plot = corner.corner(res.flatchain, labels=res.var_names,
                           truths=list(res.params.valuesdict().values()))

In [ ]:
data = {}
for i in range(len(res.var_names)):
    data[res.var_names[i]] = np.asarray(res.flatchain[res.var_names[i]])
pickle.dump(data,open('Onsala1_emcee.pkl','wb'))